# IS319 - Deep Learning

## TP3 - Recurrent neural networks

Credits: Andrej Karpathy

The goal of this TP is to experiment with recurrent neural networks for a character-level language model to generate text that looks like training text data.

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## 1. Text data preprocessing

Several text datasets are provided, feel free to experiment with different ones throughout the TP. At the beginning, use a small subset of a given dataset (for example use only 10k characters).

In [2]:
text_data_fname = 'baudelaire.txt'  # ~0.1m characters (French)
# text_data_fname = 'proust.txt'      # ~7.3m characters (French)
# text_data_fname = 'shakespeare.txt' # ~0.1m characters (English)
# text_data_fname = 'lotr.txt'        # ~2.5m characters (English)
# text_data_fname = 'doom.c'          # ~1m characters (C Code)
# text_data_fname = 'linux.c'         # ~11.5m characters (C code)

text_data = open(text_data_fname, 'r').read()
text_data = text_data[:10000] # use a small subset
print(f'Dataset `{text_data_fname}` contains {len(text_data)} characters.')
print('Excerpt of the dataset:')
print(text_data[:2000])

Dataset `baudelaire.txt` contains 10000 characters.
Excerpt of the dataset:
LES FLEURS DU MAL

par

CHARLES BAUDELAIRE


AU LECTEUR


La sottise, l'erreur, le péché, la lésine,
Occupent nos esprits et travaillent nos corps,
Et nous alimentons nos aimables remords,
Comme les mendiants nourrissent leur vermine.

Nos péchés sont têtus, nos repentirs sont lâches,
Nous nous faisons payer grassement nos aveux,
Et nous rentrons gaîment dans le chemin bourbeux,
Croyant par de vils pleurs laver toutes nos taches.

Sur l'oreiller du mal c'est Satan Trismégiste
Qui berce longuement notre esprit enchanté,
Et le riche métal de notre volonté
Est tout vaporisé par ce savant chimiste.

C'est le Diable qui tient les fils qui nous remuent!
Aux objets répugnants nous trouvons des appas;
Chaque jour vers l'Enfer nous descendons d'un pas,
Sans horreur, à travers des ténèbres qui puent.

Ainsi qu'un débauché pauvre qui baise et mange
Le sein martyrisé d'une antique catin,
Nous volons au passage un plaisir c

**(Question)** Create a character-level vocabulary for your text data. Create two dictionaries: `ctoi` mapping each character to an index, and the reverse `itoc` mapping each index to its corresponding character. Implement the functions to convert text to tensor and tensor to text using these mappings. Apply these functions to some text data.

In [33]:
# Create the vocabulary and the two mapping dictionaries
# YOUR CODE HERE
import numpy as np

idx = 0
ctoi = {}
itoc = {}

voc = set(text_data)

for elt in voc:
    ctoi[elt] = idx
    itoc[idx] = elt
    idx += 1

print(ctoi)
print(itoc)
print(len(ctoi))

# Implement the function converting text to tensor
def text_to_tensor(text, ctoi):
    # YOUR CODE HERE
    return torch.LongTensor(np.array([ctoi[c] for c in text]))


# Implement the function converting tensor to text
def tensor_to_text(tensor, itoc):
    # YOUR CODE HERE
    return ''.join([itoc[elt.item()] for elt in tensor])

# Apply your functions to some text data
# YOUR CODE HERE
#raise NotImplementedError()
a = text_to_tensor(text_data[:10], ctoi)

print(a)
print(tensor_to_text(a, itoc))



{'o': 0, 'u': 1, '-': 2, 'A': 3, 'ô': 4, ':': 5, 'F': 6, 'V': 7, 'l': 8, 'R': 9, 'h': 10, ';': 11, 'b': 12, 'N': 13, 'î': 14, ',': 15, 'â': 16, ' ': 17, 't': 18, 'S': 19, 'e': 20, 'd': 21, 'L': 22, 'D': 23, 'ù': 24, 'f': 25, 'ê': 26, 'q': 27, '»': 28, 'B': 29, 'W': 30, "'": 31, '!': 32, 'x': 33, 'Q': 34, 'p': 35, 'J': 36, 'v': 37, 'j': 38, 'à': 39, '_': 40, 'r': 41, '.': 42, '«': 43, 'T': 44, 'E': 45, '?': 46, 'O': 47, 'c': 48, 'C': 49, 'M': 50, '\n': 51, 's': 52, 'U': 53, 'è': 54, 'z': 55, 'a': 56, 'ç': 57, 'I': 58, 'H': 59, 'n': 60, 'û': 61, 'G': 62, 'é': 63, 'k': 64, 'm': 65, 'P': 66, 'g': 67, 'É': 68, 'y': 69, 'i': 70}
{0: 'o', 1: 'u', 2: '-', 3: 'A', 4: 'ô', 5: ':', 6: 'F', 7: 'V', 8: 'l', 9: 'R', 10: 'h', 11: ';', 12: 'b', 13: 'N', 14: 'î', 15: ',', 16: 'â', 17: ' ', 18: 't', 19: 'S', 20: 'e', 21: 'd', 22: 'L', 23: 'D', 24: 'ù', 25: 'f', 26: 'ê', 27: 'q', 28: '»', 29: 'B', 30: 'W', 31: "'", 32: '!', 33: 'x', 34: 'Q', 35: 'p', 36: 'J', 37: 'v', 38: 'j', 39: 'à', 40: '_', 41: 'r', 

## 2. Setup a character-level recurrent neural network

**(Question)** Setup a simple embedding layer with `nn.Embedding` to project character indices to `embedding_dim` dimensional vectors. Explain precisely how this layer works and what are its outputs for a given input sequence.

In [35]:
# YOUR CODE HERE
embedding = nn.Embedding(len(ctoi),3)

embedding(text_to_tensor(text_data[:10], ctoi))

tensor([[-1.4816,  1.5004, -0.0305],
        [ 0.6046, -1.2602,  0.9071],
        [ 0.0570, -0.0661, -0.0827],
        [-1.7510,  0.4382, -0.6317],
        [-0.9093,  1.8524,  0.9135],
        [-1.4816,  1.5004, -0.0305],
        [ 0.6046, -1.2602,  0.9071],
        [ 1.1396,  0.0655, -0.1079],
        [-1.0310,  0.3270,  0.1264],
        [ 0.0570, -0.0661, -0.0827]], grad_fn=<EmbeddingBackward0>)

**(Question)** Setup a single-layer RNN with `nn.RNN` (without defining a custom class). Use `hidden_dim` size for hidden states. Explain precisely the outputs of this layer for a given input sequence.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

YOUR ANSWER HERE

**(Question)** Create a simple RNN model with a custom `nn.Module` class. It should contain: an embedding layer, a single-layer RNN, and a dense output layer. For each character of the input sequence, the model should predict the probability of the next character. The forward method should return the probabilities for next characters and the corresponding hidden states.
After completing the class, create a model and apply the forward pass on some input text. Understand and explain the results.

*Note:* depending on how you implement the loss function later, it can be convenient to return logits instead of probabilities, i.e. raw values of the output layer before any activation function. 

In [ ]:
class CharRNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, num_layers=1):
        '''Initialize model parameters and layers.'''
        super().__init__()
        # YOUR CODE HERE
        raise NotImplementedError()

    def forward(self, tensor_data, hidden_state=None):
        '''Apply the forward pass for some text data already converted to tensor.'''
        # YOUR CODE HERE
        raise NotImplementedError()

# Initialize a model and apply the forward pass on some input text
# YOUR CODE HERE
raise NotImplementedError()

YOUR ANSWER HERE

**(Question)** Implement a simple training loop to overfit on a small input sequence. The loss function should be a categorical cross entropy on the predicted characters. Monitor the loss function value over the iterations.

In [ ]:
# Sample a small input sequence into tensor `input_seq` and store its corresponding expected sequence into tensor `target_seq`
# YOUR CODE HERE
raise NotImplementedError()

# Implement a training loop overfitting an input sequence and monitoring the loss function
def train_overfit(model, input_seq, target_seq, n_iters=200, learning_rate=0.2):
    # YOUR CODE HERE
    raise NotImplementedError()

# Initialize a model and make it overfit the input sequence
# YOUR CODE HERE
raise NotImplementedError()

**(Question)** Implement a `predict_argmax` method for your `RNN` model. Then, verify your overfitting: use some characters of your input sequence as context to predict the remaining ones. Experiment with the current model and analyze the results.

In [ ]:
class CharRNN(CharRNN):
    def predict_argmax(self, context_tensor, n_predictions):
        # Apply the forward pass for the context tensor
        # Then, store the last prediction and last hidden state
        # YOUR CODE HERE
        raise NotImplementedError()
        # Use the last prediction and last hidden state as inputs to the next forward pass
        # Do this in a loop to predict the next `n_predictions` characters
        # YOUR CODE HERE
        raise NotImplementedError()

# Initialize a model and make it overfit as above
# Then, verify your overfitting by predicting characters given some context
# YOUR CODE HERE
raise NotImplementedError()

YOUR ANSWER HERE

Using the argmax function to predict the next character can yield a deterministic generator always predicting the same characters. Instead, it is common to predict the next character by sampling from the distribution of output predictions, adding some randomness into the generator.

**(Question)** Implement a `predict_proba` method for your `RNN` model. It should be very similar to `predict_argmax`, but instead of using argmax, it should randomly sample from the output predictions. To do that, you can use the `torch.distribution.Categorical` class and its `sample()` method. Verify that your method correctly added some randomness.

In [ ]:
class CharRNN(CharRNN):
    def predict_proba(self, input_context, n_predictions):
        # YOUR CODE HERE
        raise NotImplementedError()

# Verify that your predictions are not deterministic anymore
# YOUR CODE HERE
raise NotImplementedError()

## 3. Train the RNN model on text data

**(Question)** Adapt your previous code to implement a proper training loop for a text dataset. To do so, we need to specify a sequence length `seq_len`, acting similarly to the batch size in classic neural networks. Then, you can either randomly sample sequences of length `seq_len` from the text dataset over `n_iters` iterations, or properly loop over the text dataset for `n_epochs` epochs (with a random starting point for each epoch to ensure different sequences), to make sure the whole dataset is seen by the model. Feel free to adjust training and model parameters empirically. Start with a small model and a small subset of the text dataset, then move on to larger experiments. Remember to use GPU if available.

In [ ]:
# Create the text dataset, compute its mappings and convert it to tensor
# YOUR CODE HERE
raise NotImplementedError()

# Initialize training parameters
# YOUR CODE HERE
raise NotImplementedError()

# Initialize a character-level RNN model
# YOUR CODE HERE
raise NotImplementedError()

# Setup the training loop
# Regularly record the loss and sample from the model to monitor what is happening
# YOUR CODE HERE
raise NotImplementedError()

**(Question)** From your trained model, play around with its predictions: start with a custom input sequence and ask the model to predict the rest. Analyze and comment your results.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

YOUR ANSWER HERE

## 4. Experiment with different RNN architectures

**(Question)** Experiment with different RNN architecures. Potential ideas are multi-layer RNNs, GRUs and LSTMs. All models can be extended to multi-layer using the `num_layers` parameter. Analyze and comment your results.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

YOUR ANSWER HERE